# **서울시에 있는 스타벅스 위치 분석**

## 1. 프로젝트 소개
카페 중에서 가장 유명한 '스타벅스'. 스타벅스는 입점 조건이 까다롭기로 유명하다.  
그렇다면 현재 스타벅스가 입점한 곳은 어디이며, 왜 그곳에 입점을 했을지 분석해보려고 한다.  
현재 입점한 위치 데이터는 스타벅스 홈페이지에서 웹크롤링하여 구성했다.

## 2. 목적
현재 입점한 스타벅스의 위치를 지도에 표시한 후, 그 위치들을 보고 입점 조건을 유추한다.

## 3. EDA 및 전처리 과정

In [1]:
import warnings
warnings.filterwarnings('ignore')
import requests
import folium
from pandas.io.json import json_normalize

In [2]:
# <분석 작업 순서>
# 전국(또는 특정지역)의 스타벅스 매장 위치를 찾아와서 지도위에 표시하기
# 1. requests로 스타벅스 매장 위치 데이터를 가져와서 딕셔너리 타입으로 변환한다.
# 2. json_normalize() 메소드로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# 3. folium 모듈을 사용해서 지도를 표시하고 지도위의 스타벅스 매장에 마커를 표시한다.

In [3]:
# 스타벅스 사이트에서 위치정보를 수집
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=5XLKAT2F0K'
request = requests.post(targetSite, data={
    'ins_lat'   : 37.5108295,           # 위도
    'ins_lng'   : 127.02928809999999,   # 경도
    'p_sido_cd' : '01',                 # 시,도 분류 코드
    'p_gugun_cd': '',                   # 시,군,구 분류 코드
    # 서울 전체를 보기 위해 시,도 코드에서 '01'을 넣고, 시,군,구 코드는 공란으로 한다.
    'in_biz_cd' : '' ,                  # 알 수 없는 정보(불필요)
    'iend'      : 2000,                 # 불러올 최대 매장 수
    'set_date'  : ''                    # 알 수 없는 정보(불필요)
})  

# 위 정보를 판다스 데이터프레임으로 변환
star_json = request.json() 
star_df = json_normalize(star_json, 'list')

In [4]:
# 컬럼에서 필요한 정보들만 추려 데이터프레임 재구성
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name'
                       , 'doro_address', 'lat', 'lot']]

star_df_map.info() # 결측치 없음 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        561 non-null    object
 1   sido_code     561 non-null    object
 2   sido_name     561 non-null    object
 3   gugun_code    561 non-null    object
 4   gugun_name    561 non-null    object
 5   doro_address  561 non-null    object
 6   lat           561 non-null    object
 7   lot           561 non-null    object
dtypes: object(8)
memory usage: 35.2+ KB


In [5]:
# astype() 메소드로 위도, 경도의 데이터 타입을 object에서 float으로 변경한다.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,스탈릿대치R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
556,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
557,상봉역,01,서울,0125,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.596890,127.086470
558,묵동이마트,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",37.613433,127.077484
559,묵동,01,서울,0125,중랑구,서울특별시 중랑구 동일로 952,37.615368,127.076633


## 4. folium을 이용한 지도 시각화

In [6]:
# folium을 이용하여 지도에 위치 표시
star_map = folium.Map(
    location=[star_df_map['lat'].mean(),star_df_map['lot'].mean()],
    zoom_start=12)
for (index, data) in star_df_map.iterrows():
    string = '{}점 - {}'.format(data['s_name'], data['doro_address'])
    popup = folium.Popup(string, max_width=600)
    folium.Marker(
        location=[data['lat'], data['lot']],
        popup=popup
    ).add_to(star_map)
star_map

- 시각화를 해본 결과 유독 점이 많이 모인곳은 **강남, 종로, 여의도** 이렇게 3군데로 나눌수 있다. 
- 또한 위치를 보면 **골목보단 대로변에 많이 모인것**을 직관적으로 확인할 수 있었다.

## 5. 추가적인 분석

위 지도에서 대로변에 많이 모였다고 확인했었다.  
그렇다면 지하철역 근처에도 많지 않을까? 생각이 들었다.  
실제로도 지하철역 이름을 딴 지점 이름이 많다고 생각했었는데 이번 기회에 그 수가 얼마나 되는지 찾아보겠다.

In [7]:
# 원본 데이터프레임 복사
count_df = star_df_map.copy()

In [8]:
# 지점 이름에 '역'이라는 글자가 들어간 수를 확인
count_df['s_name'].str.contains('역').value_counts()

False    401
True     160
Name: s_name, dtype: int64

'역'이 들어간 지점명이 160개, 아닌곳이 401개였다.  
하지만 '역삼'과 같이 지역 이름에 '역'이라는 글자가 있는지도 확인해야한다.  
먼저 '역'과 '역삼'이 들어간 지점 이름을 확인해보자

In [9]:
# '역' 이 들어간 지점명을 체크하기 위한 컬럼
count_df['cnt'] = count_df['s_name'].str.count('역')      

# '역삼' 이 들어간 지점명을 체크하기 위한 컬럼
count_df['yscnt'] = count_df['s_name'].str.count('역삼')

count_df

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot,cnt,yscnt
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,1,1
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223,1,0
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563,1,0
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495,0,0
4,스탈릿대치R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583,0,0
...,...,...,...,...,...,...,...,...,...,...
556,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966,1,0
557,상봉역,01,서울,0125,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.596890,127.086470,1,0
558,묵동이마트,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",37.613433,127.077484,0,0
559,묵동,01,서울,0125,중랑구,서울특별시 중랑구 동일로 952,37.615368,127.076633,0,0


In [10]:
# '역'이 들어간 지점명을 리스트로 뽑아본다.
name_list = []
for i in count_df.index:
    if count_df.iloc[i,8] >= 1:
        name_list.append(count_df.iloc[i,0])

print(len(name_list))
print(name_list)

160
['역삼아레나빌딩', '논현역사거리', '신사역성일빌딩', '봉은사역', '삼성역섬유센터R', '수서역R', '신사역', '도곡역', '한티역', '역삼초교사거리', '역삼대로', '역삼포스코', '학동역', '압구정역', '구역삼사거리', '강남구청역', '대치역', '학여울역', '선정릉역', '선릉역', '역삼럭키', '역삼역', '압구정로데오역', '역삼이마트', '미아사거리역', '수유역', '미아역', '발산역사거리', '마곡역', '우장산역', '마곡나루역', '가양역', '양천향교역', '염창역', '발산역', '까치산역', '가양역사거리', '낙성대역', '서울대역', '신림역포도몰', '서울대입구역', '어린이대공원역', '아차산역', '군자역', '강변역', '구의역', '금천구청역', '가산디지털단지역', '월계역', '노원마들역', '공릉역', '중계역', '노원역', '태릉입구역DT', '쌍문역', '남성역', '상도역', '노량진역', '신대방삼거리역', '숭실대입구역', '사당역', '홍대입구역사거리R', '홍대공항철도역', '서강광흥창역', '서강대흥역', '상수역', '홍대역', '애오개역', '망원역', '홍대역8번출구', '신촌역', '아현역', '홍제역', '충정로역', '서초역이화빌딩', '이수역사거리', '청계산입구역', '교대역', '양재역', '논현역', '이수역', '내방역', '방배역', '반포역', '신논현역', '서초역', '길음역', '월곡역', '안암역', '한성대입구역', '송파나루역DT', '송파개롱역', '문정역', '방이역', '잠실새내역', '잠실역', '석촌역', '가락시장역', '몽촌토성역', '등촌역', '목동역', '오목교역', '샛강역', '영등포역', '국회의사당역', '선유도역', '여의도역R', '문래역', '선유도역 1번출구', '영등포구청역', '서울양평역', '당산역사거리', '불광역', '연신내역사거리', '구파발역', '구산역', '혜화역', '독립문역', '동묘앞역

다행히도 '역삼'을 제외하면 지하철 역만 나왔다.  
그렇다면 '역삼'의 개수만 구해서 빼면 된다.

In [11]:
# '역삼'이 들어간 지점명을 리스트로 뽑아본다.
name_list = []
for i in count_df.index:
    if count_df.iloc[i,9] >= 1:
        name_list.append(count_df.iloc[i,0])
        
print(len(name_list))
print(name_list)

8
['역삼아레나빌딩', '역삼초교사거리', '역삼대로', '역삼포스코', '구역삼사거리', '역삼럭키', '역삼역', '역삼이마트']


역삼역을 제외하면 총 7개.  
그렇다면 서울시 전체 561개의 스타벅스중 160-7 = 153개, **약 27%가 지하철역 이름을 쓴 지점명**이다.  
  
여기에 역과는 가깝지만 중복된 지점명을 쓸 수 없어서 역이름을 쓰지 않는 지점들도 있는 것을 생각하면 최소 **3개중 1개는 지하철역과 가까운곳**임을 짐작할 수 있다.

## 6. 결론 및 제언
- 지도에서 봤을 땐 강남, 종로, 여의도에 밀집해 있었고, 이는 **회사가 많은 곳에 몰려 있음**을 알 수 있다.
- 또한 대로변에 많았는데 그중에서도 **지하철 역**에 많이 몰려 있음을 알 수 있다.
- 어쩌면 위 두 결과는 자명한 내용일수도 있다. 하지만 시각화와 수치를 통한 객관화는 **뻔한 이야기에도 뒷받침이 될 근거가 된다**는 것을 알게 되었다.